In [1]:
%matplotlib inline

import numpy as np
from numba import jit
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
import sys
from IPython import display

def print_fl(output):
    print(output)
    sys.stdout.flush()
    
T = 100.0
h = 0.01
numSteps = int(T/h)
t = 0

M = 3
n = 4*M**3
v = np.zeros((numSteps,n,3))
r = np.zeros((numSteps,n,3))
v_hat = np.zeros((numSteps,n,3))
m = 1.0
beta = 1.0
sigma_v = np.sqrt(1/(beta * m))/100

dist = np.zeros((numSteps,n,n))
F = np.zeros((numSteps,n,3))

L = 1.0

def initialise():
    global T, h, numSteps, t, M, n, v, r, v_hat, m, beta, sigma_v, F, L
    i = 0
    for mx in range(0, M):
        for my in range(0, M):
            for mz in range(0, M):
                r[t,i,0] = L*mx/M; r[t,i,1] = L*my/M; r[t,i,2] = L*mz/M
                i += 1
                r[t,i,0] = L*(mx+0.5)/M; r[t,i,1] = L*(my+0.5)/M; r[t,i,2] = L*mz/M
                i += 1
                r[t,i,0] = L*(mx+0.5)/M; r[t,i,1] = L*my/M; r[t,i,2] = L*(mz+0.5)/M
                i += 1
                r[t,i,0] = L*mx/M; r[t,i,1] = L*(my+0.5)/M; r[t,i,2] = L*(mz+0.5)/M
                i += 1
    v[t,:,:] = np.random.normal(0.0, sigma_v, size=(n,3))
    v[t,:,:] = v[t,:,:] - np.mean(v[t,:,:], axis=0)

In [2]:
initialise()

In [3]:
def calculate_distances_npmat():
    for i in range(0,n):
        dr = (r[t,0:i,:] - r[t,i,:])
        dist[t,i,0:i] = np.sqrt(np.sum(dr*dr, axis=1))
    dist[t,:,:] += dist[t,:,:].T
    
calculate_distances_npmat()

%timeit calculate_distances_npmat()

1000 loops, best of 3: 884 µs per loop


In [4]:
@jit(nopython=True)
def calculate_distances_npmat_jit_wargs(rIn, nIn):
    distOut = np.zeros((nIn, nIn))
    for i in range(0,nIn):
        dr = (rIn[0:i,:] - rIn[i,:])
        dr *= dr
        dr[:,0] += dr[:,1] + dr[:,2]
        distOut[i,0:i] = np.sqrt(dr[:,0])
    distOut[:,:] += distOut[:,:].T
    return distOut
    
calculate_distances_npmat_jit_wargs(r[t,:,:], n)

%timeit calculate_distances_npmat_jit_wargs(r[t,:,:], n)

10000 loops, best of 3: 157 µs per loop


In [5]:
@jit(nopython=True)
def intpart(x):
    return ((x // np.sign(x)) * np.sign(x)) if not x == 0.0 else 0.0

@jit(nopython=True)
def calculate_distances_loops_jit_wargs(rIn, nIn):
    distOut = np.zeros((n, n))
    for i in range(0,n):
        for j in range(0,i):
            dx = rIn[i,0] - rIn[j,0]
            dx -= intpart(dx)
            
            dy = rIn[i,1] - rIn[j,1]            
            dy -= intpart(dy)
            
            dz = rIn[i,2] - rIn[j,2]            
            dz -= intpart(dz)
            
            distOut[i,j] = np.sqrt(dx*dx + dy*dy + dz*dz)
    distOut[:,:] += distOut[:,:].T
    return distOut
    
calculate_distances_loops_jit_wargs(r[t,:,:], n)

%timeit calculate_distances_loops_jit_wargs(r[t,:,:], n)

1000 loops, best of 3: 235 µs per loop


In [21]:
@jit(nopython=True)
def calculate_distances_loops_jit_wargs(rIn, nIn):
    distOut = np.zeros((n, n))
    for i in range(0,n):
        for j in range(0,i):
            dx = rIn[i,0] - rIn[j,0]
            dx -= np.around(dx)
            
            dy = rIn[i,1] - rIn[j,1]            
            dy -= np.around(dy)
            
            dz = rIn[i,2] - rIn[j,2]            
            dz -= np.around(dz)
            
            dr = np.sqrt(dx*dx + dy*dy + dz*dz)
            if not dr == 0.0:
                distOut[i,j] = dr
            else:
                distOut[i,j] = 1e-12
    distOut[:,:] += distOut[:,:].T
    return distOut
    
calculate_distances_loops_jit_wargs(r[t,:,:], n)

%timeit calculate_distances_loops_jit_wargs(r[t,:,:], n)

10000 loops, best of 3: 66.6 µs per loop


In [6]:
@jit(nopython=True)
def calculate_forces_jit(rIn, dIn, nIn):
    Fij = np.zeros((nIn, nIn, 3))
    for i in range(0,nIn):
        for j in range(0,i):
            dr = rIn[i,:] - rIn[j,:]
            Fij[i,j,:] = dr * (48*np.power(dIn[i,j], -14) - 24*np.power(dIn[i,j], -8))
    Fij[:,:,0] += -Fij[:,:,0].T
    Fij[:,:,1] += -Fij[:,:,1].T
    Fij[:,:,2] += -Fij[:,:,2].T
    return Fij

def calculate_forces():
    global T, h, numSteps, t, M, n, v, r, v_hat, m, beta, sigma_v, F, L, dist
    dmat = calculate_distances_loops_jit_wargs(r[t,:,:], n)
    F[t,:,:] = np.sum(calculate_forces_jit(r[t,:,:], dmat, n), axis=1)
    
calculate_forces()

testF1 = F[t,:,:]

%timeit calculate_forces()

100 loops, best of 3: 2.52 ms per loop


In [7]:
@jit(nopython=True)
def calculate_forces_jit(rIn, dIn, nIn):
    Fij = np.zeros((nIn, nIn, 3))
    for i in range(0,nIn):
        for j in range(0,i):
            dr = rIn[i,:] - rIn[j,:]
            Fij[i,j,:] = dr * (48*np.power(dIn[i,j], -14) - 24*np.power(dIn[i,j], -8))
    Fij[:,:,0] += -Fij[:,:,0].T
    Fij[:,:,1] += -Fij[:,:,1].T
    Fij[:,:,2] += -Fij[:,:,2].T
    
    for i in range(0, nIn):
        for j in range(1, nIn):
            Fij[i,0,:] += Fij[i,j,:]
    
    return Fij[:,0,:]

def calculate_forces():
    global T, h, numSteps, t, M, n, v, r, v_hat, m, beta, sigma_v, F, L, dist
    dmat = calculate_distances_loops_jit_wargs(r[t,:,:], n)
    F[t,:,:] = calculate_forces_jit(r[t,:,:], dmat, n)
    
calculate_forces()

testF2 = F[t,:,:]

%timeit calculate_forces()

100 loops, best of 3: 2.67 ms per loop


In [8]:
@jit(nopython=True)
def calculate_forces_jit(rIn, dIn, nIn):
    Fij = np.zeros((nIn, nIn, 3))
    for i in range(0,nIn):
        for j in range(0,i):
            dx = rIn[i,0] - rIn[j,0]
            dy = rIn[i,1] - rIn[j,1]
            dz = rIn[i,2] - rIn[j,2]
            fLJ = 48.0*np.power(dIn[i,j], -14) - 24.0*np.power(dIn[i,j], -8)
            Fij[i,j,0] = dx * fLJ
            Fij[i,j,1] = dy * fLJ
            Fij[i,j,2] = dz * fLJ
    Fij[:,:,0] += -Fij[:,:,0].T
    Fij[:,:,1] += -Fij[:,:,1].T
    Fij[:,:,2] += -Fij[:,:,2].T
    return Fij

def calculate_forces():
    global T, h, numSteps, t, M, n, v, r, v_hat, m, beta, sigma_v, F, L, dist
    dmat = calculate_distances_loops_jit_wargs(r[t,:,:], n)
    F[t,:,:] = np.sum(calculate_forces_jit(r[t,:,:], dmat, n), axis=1)
    
calculate_forces()

testF3 = F[t,:,:]

%timeit calculate_forces()

1000 loops, best of 3: 885 µs per loop


In [9]:
@jit(nopython=True)
def calculate_forces_jit(rIn, dIn, nIn):
    Fij = np.zeros((nIn, nIn, 3))
    for i in range(0,nIn):
        for j in range(0,i):
            dx = rIn[i,0] - rIn[j,0]
            dy = rIn[i,1] - rIn[j,1]
            dz = rIn[i,2] - rIn[j,2]
            fLJ = (48.0/(dIn[i,j]**14) - 24.0/(dIn[i,j]**8))
            Fij[i,j,0] = dx * fLJ
            Fij[i,j,1] = dy * fLJ
            Fij[i,j,2] = dz * fLJ
    Fij[:,:,0] += -Fij[:,:,0].T
    Fij[:,:,1] += -Fij[:,:,1].T
    Fij[:,:,2] += -Fij[:,:,2].T
    return Fij

def calculate_forces():
    global T, h, numSteps, t, M, n, v, r, v_hat, m, beta, sigma_v, F, L, dist
    dmat = calculate_distances_loops_jit_wargs(r[t,:,:], n)
    F[t,:,:] = np.sum(calculate_forces_jit(r[t,:,:], dmat, n), axis=1)
    
calculate_forces()

testF3 = F[t,:,:]

%timeit calculate_forces()

1000 loops, best of 3: 620 µs per loop


In [10]:
@jit(nopython=True)
def calculate_forces_jit(rIn, dIn, nIn):
    Fij = np.zeros((nIn, nIn, 3))
    for i in range(0,nIn):
        for j in range(0,i):
            dx = rIn[i,0] - rIn[j,0]
            dy = rIn[i,1] - rIn[j,1]
            dz = rIn[i,2] - rIn[j,2]
            fLJ = (48.0/(dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]) - 24.0/(dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]*dIn[i,j]))
            Fij[i,j,0] = dx * fLJ
            Fij[i,j,1] = dy * fLJ
            Fij[i,j,2] = dz * fLJ
    Fij[:,:,0] += -Fij[:,:,0].T
    Fij[:,:,1] += -Fij[:,:,1].T
    Fij[:,:,2] += -Fij[:,:,2].T
    return Fij

def calculate_forces():
    global T, h, numSteps, t, M, n, v, r, v_hat, m, beta, sigma_v, F, L, dist
    dmat = calculate_distances_loops_jit_wargs(r[t,:,:], n)
    F[t,:,:] = np.sum(calculate_forces_jit(r[t,:,:], dmat, n), axis=1)
    
calculate_forces()

testF3 = F[t,:,:]

%timeit calculate_forces()

1000 loops, best of 3: 626 µs per loop


In [11]:
-4.4 // -1

4.0

In [12]:
x = -40.5

((x // np.sign(x)) * np.sign(x)) if not x == 0.0 else 0.0

-40.0